In [1]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import utils
import funcs
import eda_plots
import new_mlfuncs
import lightgbm_funcs
import dlfuncs

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
# import train data
train_filepath = 'data/merged_transformed.csv'
train_df, train_df_nulls = funcs.read_and_qa(train_filepath)

There are 40 columns and 260601 records in the dataframe.
Extracting count and percentages of nulls and non nulls


In [4]:
# import test data
test_filepath = 'data/test_values.csv'
test_df, test_df_nulls = funcs.read_and_qa(test_filepath)

There are 39 columns and 86868 records in the dataframe.
Extracting count and percentages of nulls and non nulls


In [5]:
# initial columns to drop
initial_to_drop = ['building_id']
target_var = 'damage_grade'
num_cols = ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage']
feats_to_exclude = ['has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']

In [6]:
train_red_df = new_mlfuncs.pre_ml_preprocessing(train_df, initial_to_drop, num_cols, target_var = target_var, missing_val_threshold = 40, num_cols_threshold = 0.9, low_var_threshold = 0.9)

Dropping: building_id
There are no columns below the threshold: 0.4
There are no high correlated features. No column will be dropped!
Dropping low variance feature: plan_configuration
Dropping low variance feature: legal_ownership_status


In [9]:
train_red_df.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,f,q,t,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3
1,8,900,2812,2,10,8,7,o,r,n,x,q,s,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2
2,21,363,8973,2,10,5,5,t,r,n,f,x,t,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3
3,22,418,10694,2,10,6,5,t,r,n,f,x,s,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2
4,11,131,1488,3,30,8,9,t,r,n,f,x,s,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3


In [7]:
test_red_df = new_mlfuncs.pre_ml_preprocessing(test_df, initial_to_drop, num_cols, target_var = None, missing_val_threshold = 40, num_cols_threshold = 0.9, low_var_threshold = 0.9)

Dropping: building_id
There are no columns below the threshold: 0.4
There are no high correlated features. No column will be dropped!
Dropping low variance feature: plan_configuration
Dropping low variance feature: legal_ownership_status


In [8]:
train, test, le = new_mlfuncs.target_encode_categorical_features(train_red_df, test_red_df, target=target_var, alpha=10)

In [9]:
target_var = 'target_encoded'

In [10]:
clf_f1_scr, pipe = lightgbm_funcs.lightgbm_model_1(train, target_var, feats_to_exclude, text_feats=None, stratify = True, test_size = 0.25)

Creating pipeline for LightGBM Classifier.
Fitting training data to pipeline for LightGBM Classifier.
Predicting test values for LightGBM Classifier.
Calculating accuracy score for LightGBM Classifier.
Accuracy Score for LightGBM Classifier: 0.7383002563276082
